### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Spring 2019 ###

# Stock Trading Optimization#

#### Jacob Makola (makola@wisc.edu), Finley Myers (fmyers@wisc.edu), Aaron Zhang (zzhang867@wisc.edu), and Lucas Laibly (laibly@wisc.edu)

*****

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [General Model](#2.-General)
     1. [Mathematical Model](#9.-Mathematical)
     1. [Solution](#b.-Solution)
     1. [Results](#c.-Results)
1. [Moving Average](#3.-General)
     1. [Mathematical Model](#a.-Mathematical)
     1. [Solution](#b.-Solution)
     1. [Results](#c.-Results)
1. [Regularization](#4.-Results-and-discussion)
    1. [Mathematical Model](#a.-Mathematical)
    1. [Solution](#b.-Solution)
    1. [Results](#c.-Results)
1. [Conclusion](#5.-Conclusion)

## 1. Introduction ##

The first few sentences should give a quick overview of the entire project. Then, elaborate with a description of the problem that will be solved, a brief history (with [citations](https://en.wikipedia.org/wiki/Citation) where appropriate) of how the problem came about, why it's important/interesting, and any other interesting facts you'd like to talk about. You should address and explain where the problem data is coming from (research? the internet? synthetically generated? something you have collected personally, or by surveying your friends?) Also give an outline of the rest of the report.

This section should be 300-600 words long, and **should be accessible to a general audience** (don't assume your reader has taken the class!). Although you should include references, you should also make sure that it is possible to understand the main ideas of the project and the context without clicking on any of the links or reading any of the references. Feel free to include images if you think it'll be helpful:

![fixit flowchart][flow]

For more help on using Markdown, see [this reference](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet).

[flow]: https://s-media-cache-ak0.pinimg.com/736x/f5/75/c5/f575c53b93724808c6f0211890a54900.jpg

## 2. General Model##

The general model represents the heart of our project. We looked to maximize the amount of money that could be potentially attained after one week of stock trading. Much like the traveling salesman problem, this is grossly impossible on the surface. So to make it more manageable, the assumptions we make are that:

1. We know how much the stock is going to sell for on each closing day
2. We set boundaries that a person cannot go broke (have less than 0 dollars) to buy stock and
3. Our starting amount of money on hand to buy stock is 100,000.

With these assumptions in mind, we begin to think of how to strucutre our base model. In the following program we set our first day seperate from the rest of the week tradings. This is because on the first day we have nothing to sell, and we should only buy. With the current model, we can more accurately force actions of buying and selling to occur through out the set amount of time. 

In [3]:
# How much money on hand that we have to work with
initialCash = 100000

using JuMP, Cbc, NamedArrays

# total number of days
days = [1, 2, 3, 4, 5]

# Amazon, Facebook, Mircosoft, Google, Apple -- In Order
stocks = [:am, :f, :m, :g, :ap]

# Data - One week stock value
values = NamedArray(
[1809.07 168.26 118.76 1188.04 190.35
1810.54 170.79 119.02 1184.24 192.54
1823.97 173.56 119.46 1193.36 194.93
1824.93 177.06 120.05 1206.19 195.76
1831.13 176.45 119.99 1214.68 196.57 ], (days, stocks), ("Days", "Stocks"))

m = Model(solver=CbcSolver())

# We can sell stocks
@variable(m, sell[days, stocks] >= 0,Int)

# We buy buy stock
@variable(m, buy[days, stocks] >= 0,Int)

# We can hold our current portfolio from one day to the next
@variable(m, hold[days,stocks] >= 0,Int)

# Inital amount of money at hand to purchase stock
@variable(m, cash[days] >= 0)


@constraint(m, hold_initial[j in stocks], hold[1, j] == buy[1, j])

# The balance of money that we have on hand much be equivalent to the amount we started with
# minue that of what we bought (per value)
@constraint(m, cash[1] == initialCash - sum(values[1, j] * buy[1, j] for j in stocks))

# On our first day we cannot sell anything because we dont have anything to sell
@constraint(m, sell_initial[j in stocks], sell[1, j] == 0)

# How much we buy on our first day must be less than how much money we have started with
@constraint(m, sum(values[1, j] * buy[1, j] for j in stocks) <= initialCash)

# Our hold for the current day must be equal to that of how much we held on our day previously plus how much 
# we bought for the day minus the amount we sold for the day as well. 
@constraint(m, hold_constraint[i in 2:5, j in stocks], hold[i, j] == hold[i - 1, j] + buy[i, j] - sell[i, j])

# To manage how much money we have moving from one day to next
@constraint(m, cash_constraint[i in 2:5], 
                cash[i] == cash[i - 1]  # currrent day's money on hand equal to amount from the previous day
                        + sum(values[i, j] * sell[i, j] for j in stocks) # plus our sold for the day
                        - sum(values[i, j] * buy[i, j] for j in stocks)) # minus how much we bought for the day

# How much we can buy is equal to the cost of the stock we want but must be less than how much money we have
# IMPORTANT : Our model does not supprot gambling (going into debt to buy stock in hopes of it paying off later)
@constraint(m, buy_constraint[i in 2:5], sum(buy[i, j] * values[i, j] for j in stocks) <= cash[i])

# How much we can sell depends on how much we are currently holding in our portfolio.
# We cannot sell what we do not own
@constraint(m, sell_constraint[i in 2:5, j in stocks], sell[i, j] <= hold[i - 1, j])

# On the last day, we sell everything we are holding for maximum profit in a singular trading week
@constraint(m, sum(hold[5, j] for j in stocks) == 0)

@objective(m, Max, cash[5])
solve(m)

returnVal = getobjectivevalue(m)
println("Portfolio return after one week ", returnVal)
# For some quick tips on using $\LaTeX$, 
# see [this cheat sheet](http://users.dickinson.edu/~richesod/latex/latexcheatsheet.pdf).


Portfolio return after one week 105594.70000000001


With our portfolio return value at $105,594 we have a 5%. However, this percent does not tell us what where we are putting our money, and to which stocks. 

# 2A Mathematical Model

In our mathematical model there are a few things that we need to establish prior to getting any value. These being the base cases in which our program can run.

Firstly, we need a case that states we can only hold as much as we have bought, this will take the form of:
$$H_0 = b$$ 
Where $H_0$ is our inital hold, and b represents our purchase (or buy).

Secondly, we must also establish a way of tracking our day to day cash, this will be what allows us to make a purchase of stock (negative) and what grows our money on hand (positive). Here we have write an equation that will take the form of:
$$C = C_0 - b$$ 
Where C is our cash on hand, $C_0$ is our initial cash and b is still our buy. This is crucial information to be able to assess just how much we can buy and how much we are holding at any given time.

With these in place we can begin to process our bug constraint, the one which will handle all calculations post the first day of our simulation:
$$C_i = C_{i-1} + S - B$$
$$B_c = b * d \leq C_i$$
$$S_c \leq h_{i-1}$$

$$C= \text {Cash}\ $$
$$C_i = \text {Cash on first day}\ $$ 
$$B = \text {action of buying}\ $$ 
$$B_c = \text {Buy constraint, cannot buy for more money than we have}\ $$
$$S = \text {action of selling}\ $$
$$S_c = \text {Sell constraint, cannot sell what we do not have}\ $$ 
$$h = \text {how much we are holding}\ $$
$$d = \text {values of stock}\ $$


# 2B Solution
The solution to the problem is rather exhaustive as our general solution above looks at the volatility of going from the first day to the last day. This meaning, rather than make actions from the circumstances of each day, we look at where we start and then where we end. Regardless, our solution provides us insight that we are spending every last dollar we can, other than day 4. This is where the general soltuion falls through in that it knows the future. In a real time stock market trading scenario, one does not know the future. We cover this by making the assumption at the start that the program will assume to know the future and then "in real time" depecit what to buy and when to sell. 

Going back to what occurs on Day 4 is that we want to hold as much google and apple stock as possible, now we cannot sell what we just bought so here sell our current holdings, then buy our shares into Google and Apple. Between these two companies we will have the largest profit margain at the end of Day 5.

In [9]:
# How much money we have on hand after each day of buying/selling
getvalue(cash)

cash: 1 dimensions:
[1] = 53.55999999999382
[2] = 53.55999999999381
[3] = 53.55999999999381
[4] = 52773.75
[5] = 105594.70000000001

In [5]:
# Which stocks we are buying into
getvalue(buy)

buy: 2 dimensions:
[1,:]
  [1,am] = 0.0
  [1, f] = 594.0
  [1, m] = 0.0
  [1, g] = 0.0
  [1,ap] = 0.0
[2,:]
  [2,am] = 0.0
  [2, f] = 0.0
  [2, m] = 0.0
  [2, g] = 0.0
  [2,ap] = 0.0
[3,:]
  [3,am] = 0.0
  [3, f] = 0.0
  [3, m] = 0.0
  [3, g] = 0.0
  [3,ap] = 0.0
[4,:]
  [4,am] = 0.0
  [4, f] = 0.0
  [4, m] = 0.0
  [4, g] = 43.0
  [4,ap] = 3.0
[5,:]
  [5,am] = 0.0
  [5, f] = 0.0
  [5, m] = 0.0
  [5, g] = 0.0
  [5,ap] = 0.0

In [5]:
# Which stocks we are selling on each day
getvalue(sell)

sell: 2 dimensions:
[1,:]
  [1,am] = 0.0
  [1, f] = 0.0
  [1, m] = 0.0
  [1, g] = 0.0
  [1,ap] = 0.0
[2,:]
  [2,am] = 0.0
  [2, f] = 0.0
  [2, m] = 0.0
  [2, g] = 0.0
  [2,ap] = 0.0
[3,:]
  [3,am] = 0.0
  [3, f] = 0.0
  [3, m] = 0.0
  [3, g] = 0.0
  [3,ap] = 0.0
[4,:]
  [4,am] = 0.0
  [4, f] = 594.0
  [4, m] = 0.0
  [4, g] = 0.0
  [4,ap] = 0.0
[5,:]
  [5,am] = 0.0
  [5, f] = 0.0
  [5, m] = 0.0
  [5, g] = 43.0
  [5,ap] = 2.9999999999999996

In [6]:
# How much we are holding on each day
getvalue(hold)

hold: 2 dimensions:
[1,:]
  [1,am] = 0.0
  [1, f] = 594.0000000000001
  [1, m] = 0.0
  [1, g] = 0.0
  [1,ap] = 0.0
[2,:]
  [2,am] = 0.0
  [2, f] = 594.0
  [2, m] = 0.0
  [2, g] = 0.0
  [2,ap] = 0.0
[3,:]
  [3,am] = 0.0
  [3, f] = 594.0000000000001
  [3, m] = 0.0
  [3, g] = 0.0
  [3,ap] = 0.0
[4,:]
  [4,am] = 0.0
  [4, f] = 0.0
  [4, m] = 0.0
  [4, g] = 43.0
  [4,ap] = 3.0000000000000004
[5,:]
  [5,am] = 0.0
  [5, f] = 0.0
  [5, m] = 0.0
  [5, g] = 0.0
  [5,ap] = 0.0

# 2C Results
Our results conclude that the highest area for profit comes from our last two days of trading, the build up previously is the eek out a few dollars, but the real money maker falls within Apple and Google. If we were to illogically trust this code as truth, we could make a recommendation that Apple and Google are the safest and most profitable areas to put our money, however it will come to the models following to show if that is the case or not. For now, we have a 5% profit making in 5 days, and are sitting at a 105,000$. 

## 3. Moving Average##

Here, you should code up your model in Julia + JuMP and solve it. Your code should be clean, easy to read, well annotated and commented, and it should compile! You are not allowed to use other programming languages or DCP packages such as `convex.jl`. **We will be running your code**. Having multiple code blocks separated by text blocks that explain the various parts of your solution will make it much easier for us to understand your project. You may also solve several versions of your problem with different models/assumptions. **Remember that if you do not write your description of the project and commeent your code well, we cannot understand what you have done. Even if it is technically brilliant, you will loose points if you do not write well and comment your code well.**

It's fine to call external packages such as `Gurobi`, but try to minimize the use of other packages. We want to be able to understand what is happening in your code without looking up additional references.
## a. Mathematical Model ##

## b. Solution ##


## c. Results ##

In [11]:
# this is a code block
using JuMP, Clp
m = Model(solver = ClpSolver())

things = [:horses, :donkeys, :goats]  # these are the things 
@variable(m, x[things] >= 0)          # the quantities of each of the things (can't be negative)
@constraint(m, sum(x) <= 10)          # we can't have any more than 10 things total
@objective(m, Max, x[:horses])        # we want to maximize the number of horses
solve(m)

for i in things
    println("The total number of ", i, " is: ", getvalue(x[i]))     # print result
end

The total number of horses is: 10.0
The total number of donkeys is: 0.0
The total number of goats is: 0.0


Remember to make sure your code compiles! I will be running your code!

## 4. Regularization ##

Here, you display and discuss the results. Show figures, plots, images, trade-off curves, or whatever else you can think of to best illustrate your results. The discussion should explain what the results mean, and how to interpret them. You should also explain the limitations of your approach/model and how sensitive your results are to the assumptions you made.

Use plots (see `PyPlot` examples from class), or you can display results in a table like this:

| Tables        | Are           | Cool  |
| ------------- |:-------------:| -----:|
| col 3 is      | right-aligned |\$1600 |
| col 2 is      | centered      |  \$12 |
| zebra stripes | are neat      |   \$1 |


## 5. Conclusion ##

Summarize your findings and your results, and talk about at least one possible future direction; something that might be interesting to pursue as a follow-up to your project.